In [1]:
import numpy as np
import torch

from ddnn.nn import NeuralNetwork, LinearLayer, ActivationFunction, LossFunction

## Pytorch

In [2]:
tnet = torch.nn.Sequential(
    torch.nn.Linear(8, 16),
    torch.nn.ReLU(),
    torch.nn.Linear(16, 16),
    torch.nn.ReLU(),
    torch.nn.Linear(16, 2),
)
tloss_fn = torch.nn.MSELoss()

In [3]:
tdata = torch.rand(10, 8, requires_grad=True)
tlabel = torch.rand(10, 2)
tpred = tnet(tdata)
tloss = tloss_fn(tpred, tlabel)
tloss

tensor(0.5451, grad_fn=<MseLossBackward0>)

In [4]:
tloss.backward()
tgrad = tdata.grad

## Our Implementation

In [5]:
net = NeuralNetwork(
    [
        LinearLayer((8, 16)),
        ActivationFunction(),
        LinearLayer((16, 16)),
        ActivationFunction(),
        LinearLayer((16, 2)),
    ]
)
loss_fn = LossFunction()

We force the same initial weights for both networks

In [6]:
for layer, tlayer in zip(net[:], tnet):
    if isinstance(layer, LinearLayer):
        layer.params.weights[:] = tlayer.weight.detach().numpy()
        layer.params.bias[:] = tlayer.bias.detach().numpy()

In [7]:
data = tdata.detach().numpy()
label = tlabel.numpy()
pred = net.foward(data)
loss = loss_fn.foward(pred, label)
loss

1.090184360879591

In [8]:
grad = net.backward(loss_fn.backward())

## Comparison

In [9]:
# check for correct gradient computation by comparing with pytorch
np.allclose(grad, tgrad.numpy()), np.linalg.norm(grad - tgrad.numpy())

(True, 3.0641544645357924e-09)

In [10]:
# check for correct prediction computation by comparing with pytorch
np.allclose(pred, tpred.detach().numpy()), np.linalg.norm(pred - tpred.detach().numpy())

(True, 2.787329340050096e-08)

In [11]:
for i, (layer, tlayer) in enumerate(zip(net[::-1], tnet[::-1])):
    if isinstance(layer, LinearLayer):
        assert np.allclose(layer.grads.weights, tlayer.weight.grad.detach().numpy()), i
        assert np.allclose(layer.grads.bias, tlayer.bias.grad.detach().numpy()), i